# Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set
import warnings
from pandas.io import gbq
from sklearn.model_selection import train_test_split

# Get the Data

In [166]:
%%time
query = """
            SELECT incident_number, timestamp_of_call, incident_group, stop_code_description, property_category, 
                    property_type, address_qualifier, borough_name, ward_name, 
                    first_pump_arriving_attendance_time, first_pump_arriving_deployed_from_station,
                    second_pump_arriving_attendance_time, second_pump_arriving_deployed_from_station, 
                    num_stations_with_pumps_attending, num_pumps_attending,
                    FROM `gdac-327115.LondonFire.fire_brigade`
        """

london = gbq.read_gbq(query, project_id = "gdac-327115", location="eu")

Wall time: 7.04 s


In [380]:
#Check the shape
london.shape

(32247, 16)

In [381]:
#View the first few rows
london.head()

,incident_number,timestamp_of_call,incident_group,stop_code_description,property_category,property_type,address_qualifier,postcode_district,borough_name,ward_name,first_pump_arriving_attendance_time,first_pump_arriving_deployed_from_station,second_pump_arriving_attendance_time,second_pump_arriving_deployed_from_station,num_stations_with_pumps_attending,num_pumps_attending
0,000320-01012017,2017-01-01 15:30:22+00:00,False Alarm,AFA,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,IG9,NOT GEO-CODED,Not geo-coded,<NA>,None,<NA>,None,2,2
1,003727-09012017,2017-01-09 10:41:54+00:00,False Alarm,AFA,Dwelling,House - single occupancy,Correct incident location,IG7,NOT GEO-CODED,Not geo-coded,<NA>,None,<NA>,None,2,2
2,003762-09012017,2017-01-09 12:28:39+00:00,False Alarm,AFA,Dwelling,House - single occupancy,Correct incident location,IG7,NOT GEO-CODED,Not geo-coded,<NA>,None,<NA>,None,2,2
3,006403-16012017,2017-01-16 04:02:14+00:00,False Alarm,AFA,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,IG7,NOT GEO-CODED,Not geo-coded,<NA>,None,<NA>,None,2,2
4,010947-26012017,2017-01-26 11:14:03+00:00,False Alarm,AFA,Dwelling,House - single occupancy,Correct incident location,IG10,NOT GEO-CODED,Not geo-coded,<NA>,None,<NA>,None,1,1


In [382]:
london.dtypes

incident_number                                            object
timestamp_of_call                             datetime64[ns, UTC]
incident_group                                             object
stop_code_description                                      object
property_category                                          object
property_type                                              object
address_qualifier                                          object
postcode_district                                          object
borough_name                                               object
ward_name                                                  object
first_pump_arriving_attendance_time                         Int64
first_pump_arriving_deployed_from_station                  object
second_pump_arriving_attendance_time                        Int64
second_pump_arriving_deployed_from_station                 object
num_stations_with_pumps_attending                           Int64
num_pumps_

In [383]:
#Check for missing values
london.isnull().sum()

incident_number                                   0
timestamp_of_call                                 0
incident_group                                    0
stop_code_description                             0
property_category                                 0
property_type                                     0
address_qualifier                                 0
postcode_district                                 0
borough_name                                      0
ward_name                                         0
first_pump_arriving_attendance_time            1819
first_pump_arriving_deployed_from_station      1819
second_pump_arriving_attendance_time          20281
second_pump_arriving_deployed_from_station    20281
num_stations_with_pumps_attending                68
num_pumps_attending                              68
dtype: int64

Since "second_pump_arriving_attendance_time" and "second_pump_arriving_deployed_from_station" have most of their values missing, we'll drop those from the analysis. 

In [167]:
london.drop(["second_pump_arriving_attendance_time", "second_pump_arriving_deployed_from_station"], axis=1, inplace=True)

# Split the Data into Training and Test Sets

Since we are going to be predicting if the call is a false alarm, we'll want to our test observations to be the newest in the data set. 

In [385]:
london["timestamp_of_call"].dt.year.value_counts()

2017    32247
Name: timestamp_of_call, dtype: int64

We see that all of our observations are from the same year, therefore we can just randomly split the data. However, let's first check the proportion of calls that ended up being a false alarm.  

In [386]:
london["incident_group"].value_counts()

False Alarm        15732
Special Service    10081
Fire                6434
Name: incident_group, dtype: int64

Not all incidents were either a Fire or False Alarm. There is a third category, Special Service, as well. Since the entire purpose of this project is to reduce the number of false alarms that the department wastes its resources on, we'll combine Special Services and the Fire into one category called Emergency. We could have dropped all incidents with Special Service. However, the fire department has to respond to those types of calls so it's important to include them. 

In [168]:
emergency = ["Special Service", "Fire"]

london["incident_group"].replace(emergency, "Emergency", inplace = True)

In [169]:
london["incident_group"].value_counts() / london.shape[0]

Emergency      0.512141
False Alarm    0.487859
Name: incident_group, dtype: float64

In [170]:
#Finally split the data
london_train, london_test = train_test_split(london, test_size = .15, stratify = london["incident_group"], random_state=42)

In [95]:
print(london_train.shape)
print(london_test.shape)

(27409, 14)
(4838, 14)


In [171]:
london_train.reset_index(drop=True, inplace = True)
london_train.head()

,incident_number,timestamp_of_call,incident_group,stop_code_description,property_category,property_type,address_qualifier,postcode_district,borough_name,ward_name,first_pump_arriving_attendance_time,first_pump_arriving_deployed_from_station,num_stations_with_pumps_attending,num_pumps_attending
0,023952-25022017,2017-02-25 21:25:09+00:00,Emergency,Special Service,Road Vehicle,Car,In street outside gazetteer location,IG1,REDBRIDGE,MAYFIELD,296,Ilford,1,1
1,029837-12032017,2017-03-12 01:28:04+00:00,False Alarm,AFA,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,SW1P,WESTMINSTER,ST. JAMES'S,276,Lambeth,1,2
2,041522-06042017,2017-04-06 17:03:02+00:00,Emergency,Primary Fire,Dwelling,House - single occupancy,Within same building,E16,NEWHAM,CUSTOM HOUSE,335,East Ham,2,2
3,033936-21032017,2017-03-21 10:43:54+00:00,Emergency,Special Service,Non Residential,Single shop,Correct incident location,HA8,BARNET,BURNT OAK,334,Mill Hill,1,1
4,002591-06012017,2017-01-06 15:10:11+00:00,Emergency,Special Service,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,Correct incident location,E15,NEWHAM,FOREST GATE SOUTH,99,Stratford,1,1


# Clean the Data

### Make indicator variable for Emergency outcome

In [172]:
%%capture --no-stdout 
london_train["Emergency"] = (london_train["incident_group"] == "Emergency").astype(np.int64)

In [142]:
#Change the objects variables to categorical
#london_train.loc[:, london_train.dtypes == "object"] = london_train.select_dtypes(["object"]).apply(lambda x: x.astype("category"))

### Check for missing values

In [173]:
london_train.isnull().sum()

incident_number                                 0
timestamp_of_call                               0
incident_group                                  0
stop_code_description                           0
property_category                               0
property_type                                   0
address_qualifier                               0
postcode_district                               0
borough_name                                    0
ward_name                                       0
first_pump_arriving_attendance_time          1542
first_pump_arriving_deployed_from_station    1542
num_stations_with_pumps_attending              56
num_pumps_attending                            56
Emergency                                       0
dtype: int64

### Incident Number

In [397]:
#Number of observations
london_train.shape[0]

27409

In [398]:
#Number of incidents
len(london_train["incident_number"].unique())

27409

As we'd hope, each observation in our data is a different reponse and there are no duplicates. 

### Time

In [399]:
london_train["timestamp_of_call"].head()

0   2017-02-25 21:25:09+00:00
1   2017-03-12 01:28:04+00:00
2   2017-04-06 17:03:02+00:00
3   2017-03-21 10:43:54+00:00
4   2017-01-06 15:10:11+00:00
Name: timestamp_of_call, dtype: datetime64[ns, UTC]

### Create a Month variable

In [174]:
%%capture --no-stdout
london_train["Month"] = london_train["timestamp_of_call"].dt.month

In [175]:
london_train["Month"].value_counts().reset_index().rename(columns={"index":"Month", "Month":"Count"}).sort_values(by="Month")

,Month,Count
1,1,6982
3,2,6217
2,3,6779
0,4,7431


### Create a Hour variable

In [176]:
%%capture --no-stdout
london_train["Hour"] = london_train["timestamp_of_call"].dt.hour

In [177]:
london_train["Hour"].value_counts().reset_index().rename(columns={"index":"Hour", "Hour":"Count"}).sort_values(by="Hour")

,Hour,Count
16,0,830
18,1,744
20,2,550
21,3,482
23,4,423
22,5,434
19,6,552
17,7,745
14,8,1014
12,9,1169


### Drop the timestamp variable

In [62]:
#london_train.drop("timestamp_of_call", axis = 1, inplace = True)

### Stop Code Description

In [404]:
london_train["stop_code_description"].value_counts()

AFA                                      10017
Special Service                           8580
Primary Fire                              3031
False alarm - Good intent                 2977
Secondary Fire                            2407
False alarm - Malicious                    378
Chimney Fire                                11
Flood call attended - Batch mobilised        7
Late Call                                    1
Name: stop_code_description, dtype: int64

### Property Category

In [178]:
london_train["property_category"].value_counts()

Dwelling             13048
Non Residential       6519
Road Vehicle          2369
Outdoor               2196
Outdoor Structure     1684
Other Residential     1537
Aircraft                21
Boat                    20
Rail Vehicle            15
Name: property_category, dtype: int64

In [179]:
res = ["Dwelling", "Other Residential"]
london_train["property_category"].replace(res, "Residential", inplace = True)

vehicle = ["Road Vehicle", "Aircraft", "Boat", "Rail Vehicle"]
london_train["property_category"].replace(vehicle, "Vehicle", inplace = True)

london_train["property_category"].replace("Outdoor Structure", "Outdoor", inplace = True)

C:\Users\ay887\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [180]:
london_train["property_category"].value_counts()

Residential        14585
Non Residential     6519
Outdoor             3880
Vehicle             2425
Name: property_category, dtype: int64

### Property Type

In [181]:
london_train["property_type"].value_counts()

Purpose Built Flats/Maisonettes - 4 to 9 storeys      3266
House - single occupancy                              3234
Purpose Built Flats/Maisonettes - Up to 3 storeys     2486
Car                                                   1403
Self contained Sheltered Housing                      1224
                                                      ... 
Airport - fuel storage                                   1
Minibus                                                  1
Athletics Stadium                                        1
Barn                                                     1
Intensive Farming Sheds (chickens/ pigs etc)             1
Name: property_type, Length: 256, dtype: int64

Clearly, there are way too many categories in the "property_type" variable to be used in a model. Instead of trying to logically merge categories based on group similarity, we'll group based on emergency percentage. Those categories with a high percentage of real emergencies will be grouped with other high percentage emergencies. 

In [182]:
#Group by property type and summarize by the Emergency variable
pt_grp = london_train.groupby("property_type")["Emergency"].aggregate(["mean", "sum"]).sort_values(by="mean", ascending=False)
pt_grp.reset_index(inplace = True)
#Change the proportion into a percentage
pt_grp["mean"] *= 100
#Create a new variable that ranks the values based on their percentage
pt_grp["pt_rank"] = (pt_grp["mean"] / 10).astype(np.int)
#Specifically change the values with 0 percentage to -1 as a special category i.e no chance it's a real emergency
pt_grp.loc[pt_grp["mean"] == 0, "pt_rank"] = -1
pt_grp

,property_type,mean,sum,pt_rank
0,Agricultural equipment,100.0,1,10
1,Camping tent,100.0,2,10
2,Freight plane,100.0,1,10
3,Human harm outdoors,100.0,5,10
4,Kiosk,100.0,1,10
...,...,...,...,...
251,Airport - hangar,0.0,0,-1
252,Greyhound stadium,0.0,0,-1
253,Airport - fuel storage,0.0,0,-1
254,Cricket ground,0.0,0,-1


In [183]:
#Join the london data with the pt_df to match the property type rankings
london_train = pd.merge(left = london_train, right = pt_grp[["property_type", "pt_rank"]], left_on = "property_type", right_on="property_type")

In [184]:
london_train[["property_type", "pt_rank"]].tail(n = 10)

,property_type,pt_rank
27399,Airfield/runway,10
27400,Bulk oil storage,-1
27401,Bulk oil storage,-1
27402,Stacked/baled crop,10
27403,Agricultural equipment,10
27404,Tennis Courts,10
27405,Other tent/marquee,5
27406,Other tent/marquee,5
27407,Airport - fuel storage,-1
27408,Minibus,10


In [186]:
london_train["pt_rank"].value_counts().reset_index()\
                                      .rename(columns={"index":"Property Type Rank", "pt_rank":"Number of Property Types"})\
                                      .sort_values(by="Property Type Rank")

,Property Type Rank,Number of Property Types
9,-1,189
7,0,812
1,1,5456
4,2,1437
6,3,1011
10,4,181
0,5,6974
2,6,5374
8,7,770
3,8,3926


### Address Qualifier

In [187]:
london_train["address_qualifier"].value_counts()

Correct incident location                          16392
Within same building                                4814
In street outside gazetteer location                2216
On land associated with building                    1779
In street close to gazetteer location                948
Open land/water - nearest gazetteer location         733
On motorway / elevated road                          153
Nearby address - no building in street               131
Nearby address - street not listed in gazetteer      106
In street remote from gazetteer location              96
Railway land or rolling stock                         41
Name: address_qualifier, dtype: int64

There are not too many different categories for the address qualifier variable. We'll combine a few of them based on logical relationships. 

In [188]:
street = ["In street outside gazetteer location", "In street remote from gazetteer location", 
          "In street close to gazetteer location", "Open land/water - nearest gazetteer location"]
london_train["address_qualifier"].replace(street, "Gazetter", inplace = True)

nearby = ["Nearby address - no building in street", "Nearby address - street not listed in gazetteer"]
london_train["address_qualifier"].replace(nearby, "Nearby Address", inplace = True)

other = ["On motorway / elevated road", "Railway land or rolling stock"]
london_train["address_qualifier"].replace(other, "Other", inplace = True)

In [189]:
london_train["address_qualifier"].value_counts()

Correct incident location           16392
Within same building                 4814
Gazetter                             3993
On land associated with building     1779
Nearby Address                        237
Other                                 194
Name: address_qualifier, dtype: int64

### Borough Name

In [190]:
london_train["borough_name"].value_counts()

WESTMINSTER               2114
CAMDEN                    1272
SOUTHWARK                 1215
LAMBETH                   1172
TOWER HAMLETS             1149
HACKNEY                    994
CROYDON                    980
LEWISHAM                   923
NEWHAM                     899
EALING                     898
WANDSWORTH                 890
HILLINGDON                 884
BARNET                     880
GREENWICH                  857
BRENT                      856
KENSINGTON AND CHELSEA     852
ENFIELD                    850
ISLINGTON                  848
HAMMERSMITH AND FULHAM     778
BROMLEY                    768
HARINGEY                   751
WALTHAM FOREST             742
HOUNSLOW                   699
HAVERING                   607
REDBRIDGE                  604
BEXLEY                     559
BARKING AND DAGENHAM       552
MERTON                     527
RICHMOND UPON THAMES       475
HARROW                     469
SUTTON                     461
KINGSTON UPON THAMES       419
CITY OF 

We see that there is one category, "NOT GEO_CODED" that needs to be cleaned up for the borough name variable. Let's check if it has a corresponding postcode. 

In [191]:
london_train.loc[london_train["borough_name"] == " NOT GEO-CODED", ["postcode_district", "borough_name"]]

,postcode_district,borough_name
21,IG7,NOT GEO-CODED
52,EN6,NOT GEO-CODED
78,DA1,NOT GEO-CODED
86,SL3,NOT GEO-CODED
98,IG7,NOT GEO-CODED
...,...,...
26882,RM15,NOT GEO-CODED
27059,RM14,NOT GEO-CODED
27249,SL0,NOT GEO-CODED
27388,RM20,NOT GEO-CODED


We see that the postcode won't be useful in finding the correct borough for the missing values. Let's look at the ward. 

In [192]:
london_train.loc[london_train["borough_name"] == " NOT GEO-CODED", ["ward_name", "borough_name"]].value_counts()

ward_name       borough_name  
 Not geo-coded   NOT GEO-CODED    156
dtype: int64

We also see that every missing value for borough also has a corresponding missing value for ward. Therefore, we'll just drop the "NOT GEO-CODED" from borough. 

In [193]:
%%capture --no-stdout
london_train["borough_name"].replace(" NOT GEO-CODED", np.nan, inplace=True)
london_train.dropna(subset=["borough_name"], inplace = True)

In [194]:
london_train["borough_name"].isnull().sum()

0

### Ward Name

In [195]:
london_train["ward_name"].value_counts()

WEST END                  518
ST. JAMES'S               419
FAIRFIELD                 186
MARYLEBONE HIGH STREET    177
KILBURN                   175
                         ... 
CHEAP                       6
CORNHILL                    6
BASSISHAW                   5
QUEENHITHE                  4
DOWGATE                     4
Name: ward_name, Length: 630, dtype: int64

In [196]:
%%capture --no-stdout
london_train["ward_name"].replace(" Not geo-coded", np.nan, inplace=True)
london_train.dropna(subset=["ward_name"], inplace = True)

In [197]:
london_train["ward_name"].isnull().sum()

0

In [198]:
#Group by property type and summarize by the Emergency variable
wn_grp = london_train.groupby("ward_name")["Emergency"].aggregate(["mean", "sum"]).sort_values(by="mean", ascending=False)
wn_grp.reset_index(inplace = True)
#Change the proportion into a percentage
wn_grp["mean"] *= 100
#Create a new variable that ranks the values based on their percentage
wn_grp["wn_rank"] = (wn_grp["mean"] / 10).astype(np.int)
#Specifically change the values with 0 percentage to -1 as a special category i.e no chance it's a real emergency
wn_grp.loc[wn_grp["mean"] == 0, "wn_rank"] = -1
wn_grp

,ward_name,mean,sum,wn_rank
0,ASHBURTON,91.666667,22,9
1,TUDOR,87.500000,14,8
2,DARWIN,85.714286,18,8
3,WANDSWORTH COMMON,85.714286,36,8
4,BECONTREE,84.000000,21,8
...,...,...,...,...
625,LANGBOURN,7.142857,1,0
626,WALBROOK,6.666667,1,0
627,CORDWAINER,6.250000,1,0
628,DOWGATE,0.000000,0,-1


In [199]:
#Join the london data with the pt_df to match the property type rankings
london_train = pd.merge(left = london_train, right = wn_grp[["ward_name", "wn_rank"]], left_on = "ward_name", right_on="ward_name")

In [200]:
london_train["wn_rank"].value_counts().reset_index()\
                                      .rename(columns={"index":"Ward Name Rank", "wn_rank":"Number of Wards"})\
                                      .sort_values(by="Ward Name Rank")

,Ward Name Rank,Number of Wards
10,-1,10
8,0,72
7,1,195
5,2,1960
3,3,3518
1,4,5897
0,5,7845
2,6,5244
4,7,1973
6,8,515


### First Pump Arriving Time

In [203]:
london_train["first_pump_arriving_attendance_time"].isnull().sum()

1392

In [204]:
london_train["first_pump_arriving_attendance_time"].describe()

count    25861.000000
mean       317.496346
std        135.484306
min          2.000000
25%        231.000000
50%        298.000000
75%        378.000000
max       1196.000000
Name: first_pump_arriving_attendance_time, dtype: float64

In [202]:
london_train.head()

,incident_number,timestamp_of_call,incident_group,stop_code_description,property_category,property_type,address_qualifier,postcode_district,borough_name,ward_name,first_pump_arriving_attendance_time,first_pump_arriving_deployed_from_station,num_stations_with_pumps_attending,num_pumps_attending,Emergency,Month,Hour,pt_rank,wn_rank
0,023952-25022017,2017-02-25 21:25:09+00:00,Emergency,Special Service,Vehicle,Car,Gazetter,IG1,REDBRIDGE,MAYFIELD,296,Ilford,1,1,1,2,21,8,6
1,009056-22012017,2017-01-22 01:41:22+00:00,Emergency,Special Service,Vehicle,Car,Gazetter,IG3,REDBRIDGE,MAYFIELD,194,Ilford,1,2,1,1,1,8,6
2,016512-08022017,2017-02-08 20:08:17+00:00,False Alarm,False alarm - Good intent,Vehicle,Car,Gazetter,IG3,REDBRIDGE,MAYFIELD,179,Ilford,1,3,0,2,20,8,6
3,023003-23022017,2017-02-23 19:11:48+00:00,Emergency,Primary Fire,Vehicle,Car,Gazetter,IG1,REDBRIDGE,MAYFIELD,314,Ilford,1,1,1,2,19,8,6
4,025248-01032017,2017-03-01 02:13:26+00:00,Emergency,Special Service,Vehicle,Car,Gazetter,IG3,REDBRIDGE,MAYFIELD,355,Ilford,1,1,1,3,2,8,6
